# Evaluating models


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Reading data and scoring matrix

In [8]:
import pandas as pd

In [9]:
# Read train data
train = pd.read_csv('/content/drive/MyDrive/CleanedDatasets/train.csv')

# Read scoring matrix (replace with your scoring matrix)
scoring_matrix = pd.read_csv('/content/drive/MyDrive/BT4222/BT4222_Group_3_Submission/score_matrices/scoring_matrix_ncf.csv', index_col=0)

### Set score for courses already taken by users (in training set) to 0

In [10]:
courses_taken = train.groupby('Reviewer')['Course Name'].agg(list)
user_courses_map = courses_taken.to_dict()

for reviewer, courses in user_courses_map.items():
    if reviewer in scoring_matrix.index:
        for course in courses:
            if course in scoring_matrix.columns:
                scoring_matrix.at[reviewer, course] = 0

### Evaluate

In [11]:
test_df = pd.read_csv('/content/drive/MyDrive/BT4222/BT4222_Group_3_Submission/cleaned_datasets/test.csv', index_col=0)

In [12]:
top_n = 10

In [13]:
# Get top 10 recommendations for each user
top_n_recs = scoring_matrix.apply(lambda x: list(x.nlargest(top_n).index), axis=1)

In [14]:
# Group test_df by 'Reviewer' and aggregate the courses into a list
actual_courses = test_df.groupby('Reviewer')['Course Name'].agg(list)

In [15]:
def evaluate_recommendations(recommendations, actual):
    """
    Evaluate recommendations using precision and recall for named reviewers.

    :param recommendations: Pandas Series where index is reviewer names and values are lists of recommended courses.
    :param actual: Pandas Series where index is reviewer names and values are lists of actual courses.
    :returns: Tuple containing average precision and recall for all reviewers.
    """
    precision_list = []
    recall_list = []

    # Iterate over each reviewer's recommendations and actual courses
    for reviewer, recs in recommendations.items():
        acts = actual.get(reviewer, [])
        true_positives = set(recs) & set(acts)

        precision = len(true_positives) / len(recs) if recs else 0
        recall = len(true_positives) / len(acts) if acts else 0

        precision_list.append(precision)
        recall_list.append(recall)

    average_precision = sum(precision_list) / len(precision_list) if precision_list else 0
    average_recall = sum(recall_list) / len(recall_list) if recall_list else 0

    return average_precision, average_recall

In [16]:
precision, recall = evaluate_recommendations(top_n_recs, actual_courses)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the results
print(f"Precision: {round(precision, 4)}, Recall: {round(recall, 4)}, F1 Score: {round(f1_score, 4)}")

Precision: 0.0134, Recall: 0.1167, F1 Score: 0.024
